In [12]:
import pandas as pd
from Entity.profile import Profile
from Configs.dbConfigs import FETCH_COMPANY_PEOPLE, FETCH_COMPANY_PEOPLE_FOR_OFFSET, INSERT_QUERY_PROFILE, PROFILE_TABLE_COLUMNS
from Configs.envrinomentSpecificConfgis import TABLE_NAME
from Configs.jobConfigs import COMPANY_NAME, KEYWORD_TITLE, MAX_LINKEDIN_API_CALLS_LIMIT, SEARCH_RESULT_LIMIT
from Main.login import Login
from Utils.dbUtils import insert_bulk_data, execute_get_command, readSQLQueryinPD
from Utils.utils import get_concatinated_df, get_df_items_for_search_results, get_overall_category, get_scheduler_config, get_uncommon_rows, remove_duplicates_from_df, update_schedulers_config


In [13]:
SEARCH_RESULT_LIMIT

10

In [14]:
# LOGIN TO LINKEDIN
api = Login()

# Assuming your DataFrame is named df
category_searching = get_overall_category(KEYWORD_TITLE)


In [15]:
category_searching

'hr'

In [16]:

# FETCH EXISTING DATA FROM GOOGLE SPREADSHEET
OFFSET_LIMIT = execute_get_command(FETCH_COMPANY_PEOPLE_FOR_OFFSET.format(TABLE_NAME, COMPANY_NAME.lower(), category_searching.lower()))[0][0]


In [17]:
OFFSET_LIMIT

10

In [26]:


# WE ARE ALLOWED TO MAKE AROUND 500 CALLS TO LINKEDIN PER HOURS TO PREVENT IT FROM MARKING AT AS A BOT
# WE WILL USE 350 AS PER HOUR LIMIT FOR SAFE SIDE
# TOTAL CALLS TO LIKNEDIN: LIMIT + 2 SO OUR LIMIT SHOULD NOT EXCEED 348 IN TOTAL
total_no_of_api_calls_to_lk = get_scheduler_config('total_number_of_api_calls_to_linkedin')

if (MAX_LINKEDIN_API_CALLS_LIMIT - total_no_of_api_calls_to_lk - SEARCH_RESULT_LIMIT) > 0:

    try:
        # CREATING AN EMPTY LIST TO STORE PEOPLE FROM SEARCH
        people_from_search_json_list = []

        # CALL TO SEARCH PEOPLE
        results = api.search_peoplev2(
                            keywords="Recruiters",
                            limit=SEARCH_RESULT_LIMIT,
                            offset=OFFSET_LIMIT,
                            currentCompany=[1441],
                            network=["S", "O"],
                            geoUrn=[103644278],
                            title=[],
                            resultType=["PEOPLE"],
                            # Keywords inside filters
                            company=[],
                            firstName=[],
                            lastName=None,
                            connectionOf=[],
                            followerOf=[],
                            # list of strings
                            talksAbout=[],
                            industry=[],
                            pastCompany=[],
                            profileLanguage=[],
                            schoolFilter=[],
                            schoolFreetext=[],
                            searchId=["b418ee28-c4de-4324-a085-ffe3525acddd"],
                            )

        # LOOPING OVER TO FETCH PEOPLE DATA FROM GET_PROFILE API AND PARSING RESULTS INTO OBJECT
        for peopleJSON in results:
            profile = Profile()
            
            # GET PROFILE
            peopleProfile = api.get_profile(peopleJSON["urn_id"])
            
            # PROFILE PARSING
            profile.parseProfileJSON(peopleProfile)
            profile.parseSearchPeopleJSON(peopleJSON)
            
            # APPENDING TO LIST
            people_from_search_json_list.append(get_df_items_for_search_results(profile))

        # CONVERTING LIST TO A DATAFRAME
        df_people_from_search = pd.DataFrame(people_from_search_json_list, columns=PROFILE_TABLE_COLUMNS)
        df_people_from_db = readSQLQueryinPD(FETCH_COMPANY_PEOPLE.format(TABLE_NAME, COMPANY_NAME.lower(), category_searching))
        
        # # GET THE NEW ROWS TO BE ADDED TO GOOGLE SPREADSHEET
        new_rows_added_df = get_uncommon_rows(df_people_from_db, df_people_from_search)
        # # REMOVE DUPLICATES
        no_duplicates_df = remove_duplicates_from_df(new_rows_added_df, unq_identifier_col='profile_urn_id')

        # # UPDATE DATA IN GOOGLE SPREADSHEET
        # gsheet.update_spreadsheet(no_duplicates_df)

        update_schedulers_config('total_number_of_api_calls_to_linkedin',  total_no_of_api_calls_to_lk + SEARCH_RESULT_LIMIT + 2)
    
    except e:
        print(f"Exception Occured:\n {e}")
        update_schedulers_config('total_number_of_api_calls_to_linkedin',  total_no_of_api_calls_to_lk)
else:
    print("Max api calls to linkedin exceeded. Try after an hour.")

[fetch_employees()]: OK! LinkedIn returned status code 200 (OK)
[fetch_employees()]: OK! LinkedIn returned status code 200 (OK)


D:\study-code-repeat\coding\Linkedin-Scalable-invitations\Utils\dbUtils.py:120: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(command, con=sqlConnector)


In [23]:
 new_rows_added_df = get_uncommon_rows(df_people_from_db, df_people_from_search)

In [27]:
no_duplicates_df

,public_id,profile_urn_id,profile_firstName,profile_lastName,profile_network_distance,profile_latest_company,profile_latest_job_title,category,connection_req_sent_status,connection_req_sent_date,connection_req_withdrawn_status,connection_req_withdrawn_date,record_added_to_db


In [11]:
df_people_from_db

,public_id,profile_urn_id,profile_firstName,profile_lastName,profile_network_distance,profile_latest_company,profile_latest_job_title,category,connection_req_sent_status,connection_req_sent_date,connection_req_withdrawn_status,connection_req_withdrawn_date,record_added_to_db
0,larissa-puente-8ab4a063,ACoAAA1rjwMBUWCUDiPKIrlsqqfIXyVwKFnV0ww,Larissa,Puente,3,Google,Recruiter,HR,0,None,0,None,05-06-2023
1,jeanne-skinner-4257b816,ACoAAANSZ10B-txZzruf2JZuzAlSj6poJEN1nTQ,Jeanne,Skinner,3,Google,Senior Executive Recruiter,HR,0,None,0,None,05-06-2023
2,greg-geiger-53062751,ACoAAAry7DQBRD5U_bIjfuVTndtT_MfcfpNMoOg,Greg,Geiger,2,Google,Senior Technical Recruiter,HR,0,None,0,None,05-06-2023
3,joshjenson,ACoAAAvMKwkBgXpjAzmJme0HCFIPWmhm9K4adDA,Josh,Jenson,3,Google,Recruiter,HR,0,None,0,None,05-06-2023
4,matthew-lavoie-71a9095b,ACoAAAzFXMYBgIR0U3UX8ugzOGIjhLbEvkTF0bc,Matthew,Lavoie,2,Google,Recruiter,HR,0,None,0,None,05-06-2023
5,juan-alberto-montoya-63458278,ACoAABB1VQgB-cbnQgAZ4DaL90HLHec5dY5ZHv4,Juan Alberto,Montoya,2,Google,Recruiter,HR,0,None,0,None,05-06-2023
6,sofia-grach-23723b131,ACoAACA_posBXPv9QQFXGy_vfqpNAG98I_4awFE,Sofia,Grach,3,Google,IT Recruiter,HR,0,None,0,None,05-06-2023
7,joanna-d’amico-79b1b2161,ACoAACamaOcBEBcC19ehqUUGkGVlZ920Ny8x7K8,Joanna,D’Amico,2,Google,Technical Sourcer,HR,0,None,0,None,05-06-2023
8,treshauna,ACoAACTH4pkBJqqxgEKzbk0UC93qeh0PfLymfB8,TreShauna,Gary,3,Google,Intern Recruiter,HR,0,None,0,None,05-06-2023
9,milena-staletovic-a094761a9,ACoAADCoJSkB1-UTEaFPxB38qPgFIZA8LhfD-7I,Milena,Staletovic,3,Google,Tech. Recruiter,HR,0,None,0,None,05-06-2023


In [29]:
insert_bulk_data(no_duplicates_df, INSERT_QUERY_PROFILE.format(TABLE_NAME))

TypeError: 'numpy.ndarray' object is not callable